In [3]:
from collections import defaultdict

original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
price = defaultdict(int)

for i,p in enumerate(original_price):
    price[i+1] = p

## Get the max splitting by enumerate

In [4]:
def r(n):
    return max( [price[n]] + [r(i) + r(n-i) for i in range(1,n)]    )

In [5]:
print("price[11]: ", price[11], "\t r[11]: ", r(11) )
print("price[15]: ", price[15], "\t r[15]: ", r(15) )

price[11]:  35 	 r[11]:  35
price[15]:  0 	 r[15]:  45


In [6]:
import time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

def funcTimer(func,*args):
    start = time.time()
    print(func(*args))
    print("Function running time is: ", time.time()-start)

In [7]:
funcTimer(fibonacci, 34)

5702887
Function running time is:  1.4337427616119385


In [8]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [9]:
funcTimer(fibonacci_op, 64)

14662949395604
Function running time is:  0.00020241737365722656


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [10]:
from functools import wraps
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

def f1(func):
    def wrapper(*args,**kwargs):
        print('Started Decorator')
        func(*args,**kwargs)
        print('Ended Decorator')
    return wrapper

def f():
    print('HELLO')

In [11]:
f = f1(f)
print(f.__name__)

wrapper


In [12]:
@f1
def g(a):
    print(a," from Function g()")

g('hello')

Started Decorator
hello  from Function g()
Ended Decorator


In [13]:
def k(*arg,**kwargs):
    print(kwargs)

In [14]:
k(6, b=5)

{'b': 5}


# We use this method to solve Cut Rod probelm¶

In [15]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

solution = {}

In [16]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [17]:
print(r(20))
print(price)
print(solution)

60
defaultdict(<class 'int'>, {1: 1, 2: 5, 3: 8, 4: 9, 5: 10, 6: 17, 7: 17, 8: 20, 9: 24, 10: 30, 11: 35, 15: 0, 14: 0, 13: 0, 12: 0, 20: 0, 19: 0, 18: 0, 17: 0, 16: 0})
{1: (1, 0), 2: (2, 0), 3: (3, 0), 4: (2, 2), 5: (3, 2), 6: (6, 0), 7: (6, 1), 8: (6, 2), 9: (6, 3), 10: (10, 0), 11: (11, 0), 12: (11, 1), 13: (11, 2), 14: (11, 3), 15: (13, 2), 16: (14, 2), 17: (11, 6), 18: (17, 1), 19: (17, 2), 20: (17, 3)}


# How do we parse solution?¶

In [18]:
def parse_solution(n):
    left_split, right_split = solution[n]
    if right_split == 0: return [left_split]
    return parse_solution(left_split) + parse_solution(right_split)

In [19]:
print(r(24))
print(parse_solution(20))

75
[11, 6, 3]


# Edit Distance

In [294]:
from functools import lru_cache
solution = {}

In [295]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [296]:
edit_distance('ABCDECG','ABCCEF')

3

In [297]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'A'): 'DEL G',
 ('ABCDECG', 'A

## Todo: Parse Solution is our homework

# Problem Case 3: Pinyin Auto Correction Problem

In [24]:
import pinyin
chinese_dataset = 'article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset).read()

def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [25]:
print(CHINESE_CHARATERS[:40])
print(pinyin.get('你好，中国',format='strip',delimiter=' '))
print(len(CHINESE_CHARATERS_COPYS))

此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体
ni hao ， zhong guo
129433034


In [26]:
import re
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [27]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [28]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [29]:
from collections import Counter, defaultdict
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [30]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [31]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [32]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [33]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [34]:
print(edits0('pinyin'))

{'pinyin'}


In [279]:
print(edits1('pinyin'))

{'pivnyin', 'gpinyin', 'pinyizn', 'pinhyin', 'pinoyin', 'pinyis', 'piwnyin', 'pinoin', 'pnyin', 'pignyin', 'pindyin', 'pinyivn', 'pinyino', 'pinlyin', 'pinyihn', 'pilyin', 'pinyii', 'bpinyin', 'pinynn', 'pinyinq', 'pinyyin', 'pinykin', 'pinuin', 'pinyins', 'xinyin', 'pitnyin', 'punyin', 'pcnyin', 'piqyin', 'pirnyin', 'pinyinn', 'plnyin', 'pinyinh', 'pibnyin', 'psinyin', 'pindin', 'pinlin', 'sinyin', 'pinyoin', 'pinyhin', 'pinyjin', 'ppinyin', 'pjnyin', 'ptinyin', 'pinyain', 'pinzyin', 'pinyion', 'pinjin', 'pinjyin', 'pinwin', 'pinyil', 'penyin', 'pinyuin', 'puinyin', 'pzinyin', 'pinyie', 'pinqin', 'pinvyin', 'piryin', 'pinyqin', 'pinyid', 'pinyinb', 'wpinyin', 'pinbin', 'oinyin', 'apinyin', 'pfinyin', 'lpinyin', 'hpinyin', 'phinyin', 'pniyin', 'pinyine', 'pinyinl', 'pinain', 'pinyina', 'pinkyin', 'pinydn', 'ninyin', 'spinyin', 'pinyvn', 'pjinyin', 'ypinyin', 'ipnyin', 'pinyinj', 'pigyin', 'pinhin', 'piynin', 'ppnyin', 'pkinyin', 'pinyinw', 'pinysn', 'opinyin', 'pinyi', 'pinypn', 'pyiny

# Test

In [37]:
print(correct('yin'))
print(correct('yign'))
print(correct('yinn'))

yin
ying
ying


In [38]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [40]:
print("First PINYIN: ",correct_sequence_pinyin('zhe sih yi ge ce sho'))
print("Second PINYIN: ",correct_sequence_pinyin('wo xiang shagn qinng hua da xue'))

First PINYIN:  zhe shi yi ge ce shi
Second PINYIN:  wo xiang shang qing hua da xue


# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

### n-gram

In [1]:
import re
import pinyin
def cleanText(input):
    input = re.sub('\n+', " ", input).lower()
    input = re.sub('\[[0-9]*\]', "", input)
    input = re.sub('\[[a-z]*\]', "", input)
    filtrate = re.compile(u'[^\u4E00-\u9FA5]')
    input = filtrate.sub(r' ', input)
    input = re.sub(' +', " ", input)
    return input

def chinese_to_pinyin(character,dlm=' '):
    return pinyin.get(character, format='strip', delimiter=dlm)

def trans_file_pinyin(sfile,dfile,lineCount=1000):
    with open(sfile, 'r') as f:
        fw = open(dfile, 'a')
        for i in range(lineCount):
            line = f.readline()
            line = cleanText(line)
            line = cleanText(line)
            line = chinese_to_pinyin(line)+'\n'
            line = re.sub(' +', " ", line)
            fw.write(line)
        fw.close()

In [2]:
cdset = "article_1000.txt"
ctset = "article_pinyin.txt"
trans_file_pinyin(cdset, ctset)

In [3]:
from collections import Counter, namedtuple

def ngram(docs, N=3):
    ngram_pred = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob'])
    for doc in docs:
        doc = re.sub('\n+', "", doc).lower()
        doc = re.sub(' +', " ", doc)
        spw = list(doc)
        [total_grams.append(tuple(spw[i: i+N])) for i in range(len(spw)-N+1)]
        [words.append(tuple(spw[i:i+N-1])) for i in range(len(spw)-N+2)]
        
    total_word_counter = Counter(total_grams)
    word_counter = Counter(words)
    for key in total_word_counter:
        word = ''.join(key[ : N-1])
        if word not in ngram_pred:
            ngram_pred.update({word: set()})
        next_word_prob = total_word_counter[key] / word_counter[key[:N - 1]]
        #w = Word(key[-1], '{:.3g}'.format(next_word_prob))
        w = Word(key[-1], next_word_prob)
        ngram_pred[word].add(w)
    for word, ng in ngram_pred.items():
        ngram_pred[word] = sorted(ng, key=lambda x: x.prob, reverse=True)
    return ngram_pred

def predict(text, predictions):
    text = text.lower()
    if text in predictions.keys():
        #return list(predictions[text])[:5]
        return { i.word:i.prob for i in predictions[text]}
    else:
        return {' ':0}

In [4]:
train_file = 'article_pinyin.txt'
readed_set = open(train_file,'r').readlines()
predictions = ngram(readed_set, N=2)

In [5]:
seg = input('请输入当前字符: ')
predict(seg, predictions)

请输入当前字符: p


{'i': 0.5273083081302259,
 'a': 0.24773172033446006,
 'e': 0.10531933819605052,
 'u': 0.07187333214730475,
 'o': 0.047767301191958725}

In [6]:
from collections import defaultdict
from functools import wraps

def r(n,p):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    #print("start n, p:",n, p)
    solution.append(price[n])
    pred_pinyin = None
    if n == 0:
        return p
    else:
        pred_pinyin = predict(price[n-1].lower(),predictions)
        #print(pred_pinyin)
        if price[n].lower() not in pred_pinyin.keys():
            solution.append(' ')
            #print("N: ",price[n])
            return r(n-1, p)
        elif " " not in pred_pinyin.keys():
            return r(n-1, p)
    if p*pred_pinyin[' '] > p*pred_pinyin[price[n]]:
        solution.append(' ')
        return r(n-1, p*pred_pinyin[' '])
    return r(n-1, p*pred_pinyin[price[n]])

In [7]:
mystr = "woyaoqutaikong"
price = list(mystr)

In [8]:
solution = []
proofpan = r(len(price)-1,0)

In [9]:
''.join(solution[::-1])

'wo yao qu tai kong'